In [22]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns

# Data

I performed an open ionbot and msfragger search (oxididation of M variable, max 2 missed cleavages).

See SOP document for more details on how MSFragger was ran.

In [23]:
#set this your local folder as the data was pushed to the repo as well
folder = "/home/compomics/tools/ionbot_paper"
datasets = ["PXD001468","PXD000561","PXD000815","PXD001250","TMTCPTAC"]
#datasets = ["PXD001468","PXD000815"]


 # Results
 
Load ionbot PSM results (first-matches + coeluting matches + features) + protein results.

In [24]:
def contains_missed_cleavage(x):
    if "K" in x[:-1]:
        return True
    if "R" in x[:-1]:
        return True
    return False

def is_in_s(x):
    return x in s

In [25]:
results = {}
for datas in datasets:
    for suffix in ["","_nort","_nocorr","_nocorr_nort","_closed"]:
        dataset = datas + suffix
        print(dataset)
        #Processing dataset
        if "PXD001468" in dataset:
            def get_scan_ionbot(x):
                return int(x.split('.')[1])
            def get_spectrum_file_ionbot(x):
                return x.split('.')[0]
        else:
            def get_scan_ionbot(x):
                print(x)
                return int(x.split('=')[-1])
            def get_spectrum_file_ionbot(x):
                return x.split('.')[0]
        folder2 = folder + "/" + dataset
        ionbot = pd.read_csv("%s/ionbot.first.csv"%folder2)
        ionbot["rank"] = ["first"]*len(ionbot)
        tmp = pd.read_csv("%s/ionbot.lower.csv"%folder2)
        tmp["rank"] = ["lower"]*len(tmp)
        ionbot = pd.concat([ionbot,tmp])
        #NO IDEA WHAT HAPPENED HERE...
        ionbot = ionbot.drop_duplicates("ionbot_match_id")
        tmp = pd.read_csv("%s/ionbot.features.csv"%folder2)
        ionbot = ionbot.merge(tmp,on="ionbot_match_id",how="left")
        ionbot = ionbot[(ionbot["database"]=="T")&(ionbot["q-value"]<=0.01)]
        if "PXD001468" in dataset:
            ionbot["scan"] = ionbot["spectrum_title"].apply(get_scan_ionbot)
            ionbot["sfile"] = ionbot["spectrum_title"].apply(get_spectrum_file_ionbot)
        else:
            ionbot["sfile"] = ionbot["spectrum_file"].apply(get_spectrum_file_ionbot)
        ionbot["PSM"] = ionbot["scan"].astype(str) + ionbot["sfile"] +  "XYX" +  ionbot["matched_peptide"]
        ionbot["contains_missed_cleavage"] = ionbot["matched_peptide"].apply(contains_missed_cleavage)    
        ionbot["num_passed"] = ionbot.groupby(["sfile","scan"])["scan"].transform("count")
        ionbot = ionbot[(ionbot["num_passed"]==1) |
                (ionbot["rank"]=="lower") |
                ((ionbot["num_passed"]>1) & (ionbot["rank"]=="first") & (ionbot["unexpected_modification"].isna()) & (ionbot["contains_missed_cleavage"]==False))]
        #need to re-assign best_psm
        best_psms = ionbot.sort_values("psm_score").drop_duplicates(["sfile","scan"], keep="last")
        best_psm_ids = set(best_psms["ionbot_match_id"])
        ionbot["rank"] = ionbot["ionbot_match_id"].apply(
        lambda x: "first" if x in best_psm_ids else "lower")    
        results[dataset] = ionbot


PXD001468
PXD001468_nort
PXD001468_nocorr
PXD001468_nocorr_nort
PXD001468_closed
PXD000561
PXD000561_nort
PXD000561_nocorr
PXD000561_nocorr_nort
PXD000561_closed
PXD000815
PXD000815_nort
PXD000815_nocorr
PXD000815_nocorr_nort
PXD000815_closed
PXD001250
PXD001250_nort
PXD001250_nocorr
PXD001250_nocorr_nort
PXD001250_closed
TMTCPTAC
TMTCPTAC_nort
TMTCPTAC_nocorr
TMTCPTAC_nocorr_nort
TMTCPTAC_closed


In [26]:
def is_in_s(x):
    return x in s

to_write = []

for datas in datasets:
    dataset = datas
    #compute statistics
    print(dataset)

    dataset = datas + "_closed"
    ionbot = results[dataset]
    full = len(ionbot)
    to_write.append(["","count","PSM",datas,"_closed",full])

    dataset = datas
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open",unexp])

    dataset = datas + "_nort"
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open_noR",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open_noR",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open_noR",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open_noR",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open_noR",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open_noR",unexp])

    dataset = datas + "_nocorr"
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open_no_corr",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open_no_corr",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open_no_corr",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open_no_corr",unexp])

    dataset = datas + "_nocorr_nort"
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open_noR_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open_noR_no_corr",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open_noR_no_corr",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open_noR_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open_noR_no_corr",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open_noR_no_corr",unexp])

    #overlap between open and open without corr and rt information
    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    dataset = datas + "_nocorr_nort"
    ionbot = results[dataset]
    s2 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    to_write.append(["","overlap_open_noR_no_corr","PSM",datas,"_open",len(s1.difference(s2))])
    to_write.append(["","overlap_open_noR_no_corr","PSM",datas,"intersection",len(s1.intersection(s2))])
    uniques2 = len(s2.difference(s1))
    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)
    #now I cut the peptide bacause I want the overlap in title
    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled = len(tmp2.intersection(tmp3))
    to_write.append(["","overlap_open_noR_no_corr","PSM",datas,"_open_noR_no_corr",uniques2-overruled])
    to_write.append(["overruled","overlap_open_noR_no_corr","PSM",datas,"_open_noR_no_corr",overruled])

    #overlap between open and closed
    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    dataset = datas + "_closed"
    ionbot = results[dataset]
    s2 = set(ionbot["PSM"])
    to_write.append(["","overlap_closed","PSM",datas,"intersection",len(s1.intersection(s2))])

    uniques2 = len(s2.difference(s1))
    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)
    #now I cut the peptide bacause I want the overlap in title
    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled = len(tmp2.intersection(tmp3))
    to_write.append(["","overlap_closed","PSM",datas,"_closed",uniques2-overruled])
    to_write.append(["overruled","overlap_closed","PSM",datas,"_closed",overruled])

    dataset = datas
    ionbot = results[dataset]
    s = s1
    ionbot["unique"] = ionbot["PSM"].apply(is_in_s)
    ionbot = ionbot[ionbot["unique"]==True]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","overlap_closed","PSM",datas,"_open",full-wide-unexp])
    to_write.append(["is_7Da","overlap_closed","PSM",datas,"_open",wide])
    to_write.append(["is_unexpected","overlap_closed","PSM",datas,"_open",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","overlap_closed","peptide",datas,"_open",full-wide-unexp])
    to_write.append(["is_7Da","overlap_closed","peptide",datas,"_open",wide])
    to_write.append(["is_unexpected","overlap_closed","peptide",datas,"_open",unexp])





PXD001468


/tmp/ipykernel_147128/1357946374.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
/tmp/ipykernel_147128/1357946374.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))


PXD000561
PXD000815
PXD001250
TMTCPTAC


In [27]:
to_write = pd.DataFrame(to_write,columns=["unexpected","type","level","spectrum_file","search","value"])
to_write.to_csv("id_overlap.csv",index=False)